In [ ]:
!pip install pytorch-transformers --user

In [ ]:
pip install pandas==1.3.5 --user

In [1]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset

from tqdm.notebook import tqdm
from tqdm.auto import tqdm

from transformers import BertTokenizer, DistilBertTokenizer
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import random


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
train = pd.read_pickle('./data/train.pkl')
test = pd.read_pickle('./data/test.pkl')

print("Size of the train data: ", len(train))
print("Size of the test data: ", len(test))

Size of the train data:  3464
Size of the test data:  866


In [4]:
possible_labels = train["Code"].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
print(label_dict)

{'Solution Discussion': 0, 'Expected Behaviour': 1, 'Usage': 2, 'Social Conversation': 3, 'Contribution and Commitment': 4, 'Bug Reproduction': 5, 'Motivation': 6, 'Potential New Issues and Requests': 7, 'Investigation and Exploration': 8, 'Workarounds': 9, 'Observed Bug Behaviour': 10, 'Task Progress': 11, 'Action on Issue': 12}


In [5]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [6]:
train['Code'] = train["Code"].replace(label_dict)
test['Code'] = test["Code"].replace(label_dict)

In [7]:
x_train, x_val, y_train, y_val = train_test_split(train, 
                                                  train["Code"], 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=train["Code"].values)

/cta/apps/opt/spack/linux-ubuntu18.04-cascadelake/gcc-10.2.0/anaconda3-2019.10-iuiptdanppsqsiwkt5zl63gqf6qrc5fv/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/cta/apps/opt/spack/linux-ubuntu18.04-cascadelake/gcc-10.2.0/anaconda3-2019.10-iuiptdanppsqsiwkt5zl63gqf6qrc5fv/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this wil

In [8]:
encoded_data_train = tokenizer.batch_encode_plus(
    x_train["Text Content"].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='longest',
    truncation=True, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(x_train["Code"].values)

In [9]:
encoded_data_val = tokenizer.batch_encode_plus(
    x_val["Text Content"].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='longest',
    truncation=True,
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(x_val["Code"].values)

In [10]:
from collections import Counter
count = Counter(y_train)
 
class_count = np.array([count[i] for i in count])
 
weight = 1./class_count
 
weight

array([0.00104167, 0.00389105, 0.00184162, 0.01190476, 0.00512821,
       0.004     , 0.00598802, 0.00641026, 0.01666667, 0.01785714,
       0.01176471, 0.01123596, 0.02380952])

In [11]:
from torch.utils.data import WeightedRandomSampler

samples_weight = np.array([weight[t] for t in y_train])
samples_weight=torch.from_numpy(samples_weight)

sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

In [12]:
count_val = Counter(y_val)
 
class_count_val = np.array([count_val[i] for i in count_val])
 
weight_val = 1./class_count_val

samples_weight_val = np.array([weight_val[t] for t in y_val])
samples_weight_val = torch.from_numpy(samples_weight_val)

sampler_val = WeightedRandomSampler(samples_weight_val, len(samples_weight_val))

In [13]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
print(len(dataset_train), len(dataset_val))


2944 520


In [14]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

In [15]:

batch_size = 1

dataloader_train = DataLoader(dataset_train, 
                              sampler=sampler,
                              #sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   #sampler=SequentialSampler(dataset_val), 
                                   sampler=sampler_val, 
                                   batch_size=batch_size)


In [16]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)


/cta/users/smehder/.local/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [17]:
epochs = 40

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [18]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')
    
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [19]:

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [20]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [121]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'./Models/distilbert_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.24495558464215908
Validation loss: 4.411667906487958
F1 Score (Weighted): 0.4101500557570241



Epoch 2
Training loss: 0.19517951723235694
Validation loss: 4.4773349873736095
F1 Score (Weighted): 0.48640892652168816



Epoch 3
Training loss: 0.17194548429347342
Validation loss: 4.83187087802423
F1 Score (Weighted): 0.48030841503545085



Epoch 4
Training loss: 0.18294317055702988
Validation loss: 5.869368711595395
F1 Score (Weighted): 0.47965550104198706



Epoch 5
Training loss: 0.16538170637608077
Validation loss: 4.9934420965033475
F1 Score (Weighted): 0.5313648533616151


...


Epoch 36
Training loss: 0.03922847505225629
Validation loss: 9.690344167005764
F1 Score (Weighted): 0.3751602730670165



Epoch 37
Training loss: 0.017357521815367902
Validation loss: 8.977865622065138
F1 Score (Weighted): 0.4270743441200292



Epoch 38
Training loss: 0.017648053563890577
Validation loss: 9.935456112602985
F1 Score (Weighted): 0.3558119470194741



Epoch 39
Training loss: 0.02657085419013178
Validation loss: 9.76748419769551
F1 Score (Weighted): 0.3989404441050552



Epoch 40
Training loss: 0.0251556821753401
Validation loss: 9.068009731358472
F1 Score (Weighted): 0.41917658788018736



In [39]:
checkpoint = torch.load("./Models/distilbert_epoch_30.model")
model.load_state_dict(checkpoint)
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [40]:
test['Code'] = test["Code"].replace(label_dict)
encoded_data_test = tokenizer.batch_encode_plus(
    test["Text Content"].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='longest',
    truncation=True, 
    return_tensors='pt'
)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test["Code"].values)

dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

val_loss, predictions, true_vals = evaluate(dataloader_test)
val_f1 = f1_score_func(predictions, true_vals)
tqdm.write(f'Validation loss: {val_loss}')
tqdm.write(f'F1 Score (Weighted): {val_f1}')

Validation loss: 5.8925778344789315
F1 Score (Weighted): 0.5319029720815789


In [41]:
y_pred = []
for i in range(len(predictions)):
    y_pred.append(predictions[i].tolist().index(max(predictions[i])))
print(y_pred)

[3, 8, 11, 3, 0, 8, 0, 0, 2, 3, 8, 0, 7, 6, 2, 0, 3, 0, 0, 0, 0, 5, 0, 0, 0, 3, 1, 3, 0, 5, 0, 0, 3, 0, 3, 3, 0, 0, 8, 0, 6, 0, 0, 0, 3, 0, 9, 0, 0, 0, 6, 2, 0, 5, 7, 2, 1, 2, 5, 0, 0, 7, 0, 5, 8, 3, 2, 0, 1, 0, 9, 2, 3, 5, 1, 3, 2, 3, 5, 0, 2, 3, 0, 0, 0, 2, 0, 0, 0, 6, 0, 11, 0, 3, 3, 5, 0, 3, 2, 0, 2, 1, 5, 1, 0, 6, 0, 3, 5, 2, 0, 0, 2, 2, 0, 2, 0, 7, 3, 2, 0, 0, 5, 3, 0, 3, 0, 8, 0, 0, 0, 8, 0, 0, 2, 5, 0, 0, 0, 6, 7, 0, 0, 2, 3, 0, 8, 0, 7, 3, 0, 3, 3, 0, 0, 5, 2, 2, 0, 3, 12, 5, 3, 3, 0, 2, 2, 7, 0, 8, 0, 2, 0, 11, 5, 3, 3, 0, 3, 8, 0, 2, 0, 2, 1, 0, 0, 6, 0, 6, 2, 5, 7, 3, 3, 0, 3, 6, 0, 0, 1, 0, 3, 1, 0, 10, 3, 0, 0, 0, 5, 0, 3, 2, 8, 3, 2, 5, 8, 0, 2, 8, 5, 0, 2, 0, 0, 2, 3, 0, 0, 0, 0, 2, 0, 0, 12, 0, 0, 7, 3, 3, 0, 11, 0, 3, 7, 3, 1, 0, 2, 3, 6, 3, 0, 11, 0, 8, 8, 7, 5, 3, 0, 0, 0, 0, 0, 3, 0, 0, 11, 3, 0, 0, 0, 3, 3, 2, 7, 5, 2, 5, 0, 0, 3, 0, 0, 8, 5, 0, 5, 0, 5, 2, 0, 11, 0, 0, 8, 0, 12, 2, 0, 0, 0, 2, 0, 0, 10, 0, 2, 0, 3, 0, 8, 3, 1, 8, 2, 0, 1, 3, 10, 8, 2, 3, 0, 0, 0,

In [42]:
from sklearn.metrics import precision_recall_fscore_support as score

precision, recall, fscore, support = score(true_vals, y_pred)

for label, i in label_dict.items():
    print('{}: \t\tprecision: {:.2f}, recall: {:.2f}, fscore: {:.2f}, support: {}'.format(label, precision[i], recall[i], fscore[i], support[i]))

Solution Discussion: 		precision: 0.57, recall: 0.72, fscore: 0.64, support: 282
Expected Behaviour: 		precision: 0.24, recall: 0.32, fscore: 0.28, support: 25
Usage: 		precision: 0.39, recall: 0.54, fscore: 0.45, support: 74
Social Conversation: 		precision: 0.80, recall: 0.74, fscore: 0.77, support: 159
Contribution and Commitment: 		precision: 0.67, recall: 0.35, fscore: 0.46, support: 17
Bug Reproduction: 		precision: 0.42, recall: 0.41, fscore: 0.41, support: 49
Motivation: 		precision: 0.44, recall: 0.24, fscore: 0.31, support: 58
Potential New Issues and Requests: 		precision: 0.33, recall: 0.28, fscore: 0.31, support: 46
Investigation and Exploration: 		precision: 0.48, recall: 0.33, fscore: 0.39, support: 75
Workarounds: 		precision: 0.50, recall: 0.11, fscore: 0.18, support: 18
Observed Bug Behaviour: 		precision: 0.33, recall: 0.15, fscore: 0.21, support: 26
Task Progress: 		precision: 0.55, recall: 0.48, fscore: 0.51, support: 25
Action on Issue: 		precision: 0.70, recall: 